# 0. Librerías

In [56]:
import pandas as pd
import numpy as np
import cufflinks as cf
from re import sub, UNICODE
from unicodedata import normalize
from emoji import demojize
from nltk.corpus import stopwords
cf.go_offline()

# 1. Entendimiento del problema de negocio y de los datos

## 1.1. Importación de Datos

In [23]:
data = pd.read_csv('/Users/efraflores/Desktop/EF/EF/Practice/data_app_movil.csv')
data

,Unnamed: 0,at,content,reviewCreatedVersion,thumbsUpCount,content.1,topic
0,6,2021-09-22 23:45:00,"Que paso con la app, al abrir se cierra despué...",5.62,217,"Que paso con la app, al abrir se cierra despué...",transferencias
1,9,2021-03-21 17:26:06,"No funciona, después de más de una hora entre ...",5.59,201,"No funciona, después de más de una hora entre ...",transferencias
2,10,2021-08-04 14:39:20,"La app ha intentado mejorar desde el 2019, per...",5.60,185,"La app ha intentado mejorar desde el 2019, per...",transferencias
3,19,2021-06-14 16:44:16,"Poner una estrella es mucho, sólo porque lo pi...",5.59.2,131,"Poner una estrella es mucho, sólo porque lo pi...",transferencias
4,24,2021-07-02 14:36:40,Esta aplicación no deja de presentar fallas de...,5.60,126,Esta aplicación no deja de presentar fallas de...,transferencias
...,...,...,...,...,...,...,...
7461,99794,2021-06-14 12:39:10,"No puedo hacer transferencias, ni puedo consul...",5.59.2,0,"No puedo hacer transferencias, ni puedo consul...",token
7462,99901,2021-06-15 18:09:27,Muchos problemas para activar el Token y probl...,5.59.2,0,Muchos problemas para activar el Token y probl...,token
7463,99921,2021-06-15 09:49:32,Quiero hacer trafercias a otros bancos y nisiq...,5.59.2,0,Quiero hacer trafercias a otros bancos y nisiq...,token
7464,99936,2021-06-15 00:44:27,4 estrellas por no poder dar d alta el token,NaN,0,4 estrellas por no poder dar d alta el token,token


In [24]:
dupli = data[data['Unnamed: 0'].duplicated()]['Unnamed: 0'].tolist()
dupli_df = data[data['Unnamed: 0'].isin(dupli)].copy()
dupli_df.shape

(1108, 7)

In [28]:
df = data.loc[~data.index.isin(dupli_df.index),:].copy()
df.shape

(6358, 7)

In [31]:
df[df['Unnamed: 0'].duplicated()]

,Unnamed: 0,at,content,reviewCreatedVersion,thumbsUpCount,content.1,topic


In [30]:
dupli_df.shape[0]+df.shape[0]

7466

## 1.2. Entendimiento del problema de negocio

* Hacer análisis de texto del content
* Poder analizar en el texto si el cliente se refirió a transferencia o token
* Identificar los parecidos a transferencias para no disminuir los datos
    * Ejemplos: SuperToken = token  ,  trasfercias = transferencias 
* Construir un Modelos de Aprendizaje Máquina para predecir de forma correcta el tópico de un texto y mandar con ello al equipo adecuado
* Evaluar estos Modelos con un set de Test
* Elegir a producción un Modelo de preferencia simple y con buen Trade off entre Sesgo y Varianza


# 2. Exploración de datos

* Nota: Se notó que content = content.1 por lo que solo usaremos 'content' como base y 'content.1' como copia de respaldo.

## 2.1. Exploración: Longitud, Tipo de datos, Identificación de Nulos e Identificación de Duplicados

### Longitud:

In [24]:
data.shape

(7466, 7)

### Tipo de datos:

In [25]:
data.dtypes
# Aquí tenemos que corregir los tipos de datos
# Y la variable Unnamed: 0 , paracería no ser informativa hasta el momento


Unnamed: 0               int64
at                      object
content                 object
reviewCreatedVersion    object
thumbsUpCount            int64
content.1               object
topic                   object
dtype: object

#### Tratamiento: Transformando los datos a su correspondiente tipo

* La única variable que no esta en su correspondiente tipo es 'at' pues esta debe ser de tipo date. 
Las demás si deben ser str o int, cabe resaltar que 'reviewCreatedVersion' si debe ser de tipo object o str
pues, existen versiones como por ejemplo la '5.59.2'

In [32]:
#Transformando
data['at'] = pd.to_datetime(data['at'], format="%Y-%m-%d %H:%M:%S")

In [38]:
#Comprobando
data['at'].dtypes
# este es un tipo de formato fecha

dtype('<M8[ns]')

### Identificación de Nulos:


In [27]:
data.isnull().sum()/len(data)
# Se identifica que no hay muchos valores ausentes, lo que es bueno, y los pocos que hay son de la variable 
# "reviewCreatedVersion", su imputación se tratará en la sección "3.2. Tratamiento de Nulos"

Unnamed: 0              0.000000
at                      0.000000
content                 0.000000
reviewCreatedVersion    0.052907
thumbsUpCount           0.000000
content.1               0.000000
topic                   0.000000
dtype: float64

### Identificación de Duplicados:

* Unnamed: 0 resultó ser informativa, pues nos ayuda a identificar duplicados

In [57]:
#Estos son algunos "Duplicados"
data[data['Unnamed: 0'].duplicated(keep=False)].sort_values(by=['Unnamed: 0'])[0:12]

,Unnamed: 0,at,content,reviewCreatedVersion,thumbsUpCount,content.1,topic
8,33,2020-11-01 01:17:37,Muchos problemas con Super Token para el cambi...,5.58.1,108,Muchos problemas con Super Token para el cambi...,transferencias
4644,33,2020-11-01 01:17:37,Muchos problemas con Super Token para el cambi...,5.58.1,108,Muchos problemas con Super Token para el cambi...,token
4646,36,2021-09-13 12:05:26,Pésima app. Calidad muy mala. Se sale de la ap...,5.61,102,Pésima app. Calidad muy mala. Se sale de la ap...,token
9,36,2021-09-13 12:05:26,Pésima app. Calidad muy mala. Se sale de la ap...,5.61,102,Pésima app. Calidad muy mala. Se sale de la ap...,transferencias
11,39,2021-03-22 11:28:35,La aplicación funcionaba correctamente hasta q...,5.59,98,La aplicación funcionaba correctamente hasta q...,transferencias
4648,39,2021-03-22 11:28:35,La aplicación funcionaba correctamente hasta q...,5.59,98,La aplicación funcionaba correctamente hasta q...,token
17,63,2021-03-23 18:29:17,"Es sencilla de usar, pero es fastidioso el hec...",5.59.1,77,"Es sencilla de usar, pero es fastidioso el hec...",transferencias
4653,63,2021-03-23 18:29:17,"Es sencilla de usar, pero es fastidioso el hec...",5.59.1,77,"Es sencilla de usar, pero es fastidioso el hec...",token
21,78,2021-09-03 15:58:46,Se tarda mucho en cargar. Fallan mucho las tra...,5.60,68,Se tarda mucho en cargar. Fallan mucho las tra...,transferencias
4656,78,2021-09-03 15:58:46,Se tarda mucho en cargar. Fallan mucho las tra...,5.60,68,Se tarda mucho en cargar. Fallan mucho las tra...,token


In [50]:
# Este es el número de duplicados
data[data['Unnamed: 0'].duplicated(keep=False)].sort_values(by=['Unnamed: 0']).shape[0] / 2

554.0

* Super Nota: Se encontró un "patrón" en los "Duplicados" tienen un sentido pues 
quieren decir que un mismo cliente tuvo ambos problemas, el problema del "Token" y de la "Transferencia"
* Por tanto, NO se eliminan los Duplicados

## 2.2. Visualización de Datos con la siguiente estructura: Pregunta, Conocimiento de los datos, Tabla y Gráfica 

### Pregunta 1. ¿En qué fecha escribieron más los clientes?

* Conocimiento de los datos

In [62]:
print('Los datos van del:')
print(str(data['at'].max())+' al '+str(data['at'].min()))

Los datos van del:
2021-10-22 23:16:15 al 2020-08-08 01:52:32


In [37]:
df['at'] = df['at'].map(pd.to_datetime)
df['date'] = df['at'].dt.date

In [43]:
df.assign(n=1).pivot_table(index='date', values='n', aggfunc='count').iplot()

* Tabla

In [88]:
# Se filtrará por Mes y Año, y se mostrarán las frecuencias
months = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
years = ['2020','2021']
table_date = pd.DataFrame(np.random.randn(12, 2),index=months,columns=years)
for val in years:
    for i in range(0,len(months)):
        table_date[val][i] = data[(data['at'].dt.month == i+1) & (data['at'].dt.year == int(val))].shape[0] 
table_date

#data[data['at'].dt.month == 11] 

,2020,2021
Enero,0.0,402.0
Febrero,0.0,334.0
Marzo,0.0,762.0
Abril,0.0,440.0
Mayo,0.0,431.0
Junio,0.0,375.0
Julio,0.0,362.0
Agosto,366.0,842.0
Septiembre,466.0,735.0
Octubre,416.0,553.0


In [86]:
data[(data['at'].dt.month == 8) & (data['at'].dt.year == 2020)].shape 

(366, 7)

# 3. Preprocesamiento de Datos

## 3.1. Transformar los tipos de datos a su correspondiente y eliminar variables no informativas

* Nota: esta parte ya se trato en la sección "2.1 - Tipo de Datos" pues era necesario hacerlo desde antes para hacer una buena exploración de datos

## 3.2. Nulos

## 3.3. Texto

### Limpieza

In [57]:
def clean_text(text: str, pattern: str="[^a-zA-Z\s]", lower: bool=True, emoji: bool=True, to_singular: bool=True) -> str: 
    '''
    Limpieza de texto
    '''
    # "Traduce" emojis, ej: 🇲🇽 --> :Mexico:
    if emoji: text = demojize(text)
    # Reemplazar acentos: áàäâã --> a
    clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
    # Omitir caracteres especiales !"#$%&/()=...
    clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)
    # Mantener sólo un espacio
    clean = sub(r'\s{2,}', ' ', clean.strip())
    # Minúsculas si el parámetro lo indica
    if lower: clean = clean.lower()
    # Omitir la última "s" si el parámetro lo indica
    if to_singular: clean = sub('s\s',' ',clean+' ')
    # Omitir las stopwords indicadas
    clean = ' '.join([x for x in clean.split() if x not in stopwords.words('spanish')])
    # Si el registro estaba vacío, indicar nulo
    if clean in ('','nan'): clean = np.nan
    return clean

In [58]:
df['content_limpio'] = df['content'].map(clean_text)
df[['content','content_limpio']].sample(3)

,content,content_limpio
1681,De repente se traba y no deja realizar transfe...,repente traba deja realizar transferencia visu...
1271,Solo a la hora de agregar una tarjeta o cuenta...,solo hora agregar tarjeta cuenta banco tarda m...
3519,"Tengo días intentando transferir, sigo instruc...",dia intentando transferir sigo instruccione re...


### Características del texto

In [72]:
df['longitud'] = df['content_limpio'].str.len()
df['palabras'] = df['content_limpio'].str.split().str.len()
df['relevancia'] = df['longitud'] / df['content'].str.len()
df['is_transf'] = df['content_limpio'].map(lambda x: 'tra' in x)
df['is_token'] = df['content_limpio'].map(lambda x: 'token' in x)
df.sample()

,Unnamed: 0,at,content,reviewCreatedVersion,thumbsUpCount,content.1,topic,date,content_limpio,longitud,palabras,relevancia,is_transf,is_token
7420,98245,2021-06-23 13:26:38,"Cambie de equipo movil, más no de número y no ...",5.59.2,0,"Cambie de equipo movil, más no de número y no ...",token,2021-06-23,cambie equipo movil ma numero deja activar sup...,76,12,0.730769,False,True


In [78]:
aux = df.assign(n=1).pivot_table(index='topic', columns='is_token', aggfunc={'n':'count'})
aux /= aux.sum().sum()
aux

n          
is_token           False     True 
topic                             
token           0.006606  0.350739
transferencias  0.640610  0.002045

In [90]:

# Ingeniería de variables
from numpy import array
from pandas import DataFrame, options
options.mode.chained_assignment = None

# Modelos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

def train_model(X: DataFrame, y: array, scaler=RobustScaler, model=LinearRegression, rem_outliers: bool=False, perc_outliers: float=0.03, **kwargs): 
        '''
        Escala y entrena un modelo, devuelve el score, el objeto tipo Pipeline y la relevancia de cada variable
        '''
        if rem_outliers:
            # Unir la tabla
            df = X.join(DataFrame(y, index=X.index, columns=['real']))
            # Y nuevamente separar la matriz X y el vector "y"
            X = df[[x for x in df.columns if x not in ['real']]].copy()
            y = df['real'].values

        # Conjunto de entrenamiento y de test
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, random_state=7, shuffle=True)

        # Define los pasos del flujo
        pipe_obj = Pipeline(steps=[('prep', scaler()), ('model', model(**kwargs))])

        # Entrena y guarda el score en test
        test_score = pipe_obj.fit(X_train,y_train).score(X_test, y_test)
        # Guarda el score en train, para revisar sobreajuste
        train_score = pipe_obj.score(X_train,y_train)

        # Imprime los scores
        print(f"Test score:\t{'{:.2%}'.format(test_score)}\nTrain score:\t{'{:.2%}'.format(train_score)}")

        # Elige la forma de obtener las variables más representativas
        # Ya sea por Regresión Lineal
        try: most_important_features = pipe_obj[-1].coef_ 
        except: 
            # O por Árbol de decisión, Bosque Aleatorio, XGBoost
            try: most_important_features = pipe_obj[-1].feature_importances_
            # De otro modo, solamente asignar un vector de 0s a este objeto
            except: most_important_features = [0]*len(X.columns)

        # Las ordena descendentemente
        coef_var = DataFrame(zip(X.columns, most_important_features)).sort_values(1, ascending=False).reset_index(drop=True)
        coef_var = coef_var.rename(columns={0:'Variable',1:'Peso'}).set_index('Variable')

        # Devuelve el objeto para clustering, la lista de scores tanto en train como en test y la relevancia de cada variable para el modelo 
        return pipe_obj, (test_score,train_score), coef_var

In [94]:
X = df[['longitud', 'palabras','relevancia', 'is_transf', 'is_token']].copy()
y = df['topic'].map(lambda x: x=='token').values

In [95]:
model_linreg, scores_linreg, coef_linreg = train_model(X, y)

Test score:	94.73%
Train score:	96.91%


In [98]:
coef_linreg

,Peso
Variable,
is_token,0.903332
palabras,0.018424
relevancia,-0.000492
longitud,-0.014927
is_transf,-0.102827


In [96]:
from sklearn.ensemble import RandomForestClassifier

model_forest, scores_forest, coef_forest = train_model(X, y, model=RandomForestClassifier)

Test score:	99.37%
Train score:	100.00%


In [97]:
from sklearn.neural_network import MLPClassifier

model_neural, scores_neural, coef_neural = train_model(X, y, model=MLPClassifier)

Test score:	99.37%
Train score:	99.69%


# 4. Entrenamiento de modelos (al menos 3)

# 5. Evaluación de los modelos

# 6. Selección del mejor modelo